NREL WIND is a federal database containing extremely detailed weather data for 2007-2012 accessible through a HSDS API.
https://developer.nrel.gov/docs/wind/wind-toolkit/

In [1]:
%matplotlib inline
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pyproj
from pyproj import Proj
import dateutil
import pymongo as pm
import pyprind
import gridfs
import json
import pickle
import geopandas as gpd
import os

In [32]:
h5pyd.getServerInfo()

h5pyd.Config()

f = h5pyd.File("/nrel/wtk-us.h5", 'r')

{'greeting': 'Welcome to HSDS!',
 'about': 'HSDS is a webservice for HSDS data',
 'name': 'NREL Highly Scalable Data Service (HSDS)',
 'start_time': 1534185577,
 'hsds_version': '0.1',
 'state': 'READY',
 'endpoint': 'https://developer.nrel.gov/api/hsds/',
 'username': 'None',
 'password': '****'}

In [6]:
#load our datasets

windspeed = f['windspeed_10m']
winddirection = f['winddirection_10m']
precipitation = f['precipitationrate_0m'] 
relative_humidity = f['relativehumidity_2m']
temp = f['temperature_2m']
dt = f["datetime"]
dt = pd.DataFrame({"datetime": dt[:]},index=range(0,dt.shape[0]))
dt['datetime'] = dt['datetime'].apply(dateutil.parser.parse)

In [7]:
dt.tail()

,datetime
61363,2013-12-31 19:00:00
61364,2013-12-31 20:00:00
61365,2013-12-31 21:00:00
61366,2013-12-31 22:00:00
61367,2013-12-31 23:00:00


In [8]:
twentytwelve = dt.loc[(dt.datetime >= '2012-01-01') & (dt.datetime < '2013-01-01')].index
twentytwelve

Int64Index([43824, 43825, 43826, 43827, 43828, 43829, 43830, 43831, 43832,
            43833,
            ...
            52598, 52599, 52600, 52601, 52602, 52603, 52604, 52605, 52606,
            52607],
           dtype='int64', length=8784)

In [9]:
timestep = dt.loc[dt.datetime == '2012-04-01 12:00:00'].index[0]
timestep

46020

NREL functions for going from lat-lon to WIND internal coordinates, which are 1 km squares.

In [10]:
def indicesForCoord(f, lat_index, lon_index):
    dset_coords = f['coordinates']
    projstring = """+proj=lcc +lat_1=30 +lat_2=60 
                    +lat_0=38.47240422490422 +lon_0=-96.0 
                    +x_0=0 +y_0=0 +ellps=sphere 
                    +units=m +no_defs """
    projectLcc = Proj(projstring)
    origin_ll = reversed(dset_coords[0][0])  # Grab origin directly from database
    origin = projectLcc(*origin_ll)
    
    coords = (lon_index,lat_index)
    coords = projectLcc(*coords)
    delta = np.subtract(coords, origin)
    ij = [int(round(x/2000)) for x in delta]
    return tuple(reversed(ij))

def indicesForBBox(f, lat_range, lon_range):
    xmin = None
    xmax = None
    ymin = None
    ymax = None
    for i in [0,1]:
        for j in [0,1]:
            yx = indicesForCoord(f,lat_range[i],lon_range[j])
            if xmin is None or yx[1] < xmin:
                xmin = yx[1]
            if xmax is None or yx[1] > xmax:
                xmax = yx[1]
            if ymin is None or yx[0] < ymin:
                ymin = yx[0]
            if ymax is None or yx[0] > ymax:
                ymax = yx[0]
            if xmin == xmax:
                xmax += 1
            if ymin == ymax:
                ymax +=1
    return ([xmin,xmax],[ymin,ymax])

In [15]:
df = pickle.load(open('2012_fire_bounds.pkl', 'rb'))
df.head()

,fire_id,left,right,top,bottom,x,y,a
2,ID-SCF-G4A0,-1635840.0,-1390080.0,2711040.0,2257920.0,8192.0,15104.0,1.113588e+11
0,WA-OWF-G70S,-1912320.0,-1712640.0,3064320.0,2872320.0,6656.0,6400.0,3.833856e+10
26,VA-VAF-GR0U,1359360.0,1505280.0,1904640.0,1720320.0,4864.0,6144.0,2.689597e+10
7,MN-MNS-G85U,-76800.0,76800.0,2895360.0,2764800.0,5120.0,4352.0,2.005402e+10
64,MT-MCD-G0QM,-867840.0,-760320.0,2603520.0,2488320.0,3584.0,3840.0,1.238630e+10


In [17]:
epsg5070 = pyproj.Proj("+init=epsg:5070")
epsg4269 = pyproj.Proj("+init=epsg:4269")

-1635840.0 2711040.0
-117.18020922476201 45.73998841207507


NREL WIND only allows 500 requests an hour, so this takes two sets of requests to run.  We iterate through the unique fires database, check against the `done` set, and then dump to a pickle file.

In [ ]:
unique_fires =  set(df.fire_id.unique())
print(len(unique_fires))

done =  set(os.listdir('wind'))
    
unique_fires = unique_fires-done

print(len(unique_fires))

In [77]:


a= [x for x in np.arange(len(df)-1, -1, -1)]

for k in a:
    fire = df.iloc[k] #this is not the preferred way to access a df, but it works
    
    fire_id = fire.fire_id
    
    done = os.listdir('wind')
    if fire_id +' wind.pkl' not in done:

        lon1=df.iloc[k,1]
        lat1=df.iloc[k,3]

        lon2 = df.iloc[k,2]
        lat2 = df.iloc[k,4]

        xs = []
        ys = []

        x,y = pyproj.transform(epsg5070, epsg4269, lon1, lat1)
        xs.append(x)
        ys.append(y)
        x,y = pyproj.transform(epsg5070, epsg4269, lon1, lat2)
        xs.append(x)
        ys.append(y)
        x,y = pyproj.transform(epsg5070, epsg4269, lon2, lat1)
        xs.append(x)
        ys.append(y)
        x,y = pyproj.transform(epsg5070, epsg4269, lon2, lat2)
        xs.append(x)
        ys.append(y)

        bounds = [min(ys), max(ys), min(xs), max(xs)]

        x_range, y_range =indicesForBBox(f, (bounds[0], bounds[1]), (bounds[2], bounds[3]))

        fire_dates =  df2[df2.uniquefire==fire_id]    
        enddate =  fire_dates.iloc[0].date
        startdate = fire_dates.iloc[-1].date

        (startdate, enddate)

        t0 = dt.loc[dt.datetime == startdate].index[0]-100
        tf = dt.loc[dt.datetime == enddate].index[0]

        local_ws = windspeed[t0:tf, y_range[0]:y_range[1], x_range[0]: x_range[1]]

        local_wd = winddirection[t0:tf, y_range[0]:y_range[1], x_range[0]: x_range[1]]

        local_precip = precipitation[t0:tf, y_range[0]:y_range[1], x_range[0]: x_range[1]]

        local_humidity = relative_humidity[t0:tf, y_range[0]:y_range[1], x_range[0]: x_range[1]]

        local_temp = temp[t0:tf, y_range[0]:y_range[1], x_range[0]: x_range[1]]

        shape = local_ws.shape

        data = {
                'fire_id':fire_id,
                'startdate': startdate,
                'enddate': enddate,
                'shape':  shape,
                'bounds' : bounds,
                'windspeed' : local_ws,
                'winddirection' : local_wd,
                'precipitation': local_precip,
                'humidity' : local_humidity,
                'temperature' : local_temp
                }

        outfile = open('wind/'+ fire_id +' wind.pkl','wb')
        pickle.dump(data,outfile)
        outfile.close()

CA-NOD-G4Z9
OR-BUD-G1G1
UT-SCS-GZT8
WA-SPD-G70Y
MT-CRA-G4MJ
NE-NES-G3HX
ID-NPF-000368
ID-CWF-G41C
ID-BOF-G4QD
NE-NES-G7A5
MT-NCA-GY9W
NV-EKD-G5LM
ID-NPF-G42D
CA-NOD-G5Z8
ID-SCF-G34R
NM-GNF-GU3S
OR-VAD-G1HG
ID-CWF-G69S
NV-WID-G4ZC
ID-STF-G5DR
MT-EAS-G4GD
WA-OWF-G72P
MT-MCD-G0QM
MN-MNS-G85U
VA-VAF-GR0U
WA-OWF-G70S
ID-SCF-G4A0
